In [1]:
import numpy as np
import h5py as h5
import sys
import physical_data as const
from read_parameter_file import read_params
import reading_data_for_los as RD
import matplotlib.pyplot as plt
from computeibb import computeibb as computeib
from prodata import projectdata

This is saying the use snapshot file is True


In [2]:
#Load dummy parameters from a "typical" specwizard run 
parameters = read_params('dummy.par')

#Load a small EAGLE box for testing
fname = "/home/andres/small_snap/snap_028_z000p000"

#Load a example of a postprocess LOS file
fname2 = "/home/andres/part_los_z0.010"

#Load class that contianes the particle information i,e position, velocities etc.
loss = RD.read_particle_data_from_los(fname2,0)        

#Load the class the contains the header information
simdata = RD.read_header(fname2)

#Conversion factors 
convert = RD.units_and_factors_for_los(fname2)

# This outputs which is what is the long axis from the LOS  (0=x,1=y,z=2)
los_long_axis = np.argmax([(loss.Position[:,0].max()-loss.Position[:,0].min()) /simdata.HubbleParam * simdata.ExpansionFactor,(loss.Position[:,1].max()-loss.Position[:,1].min()) /simdata.HubbleParam * simdata.ExpansionFactor,(loss.Position[:,2].max()-loss.Position[:,2].min()) /simdata.HubbleParam * simdata.ExpansionFactor])

LOS0/PartType0/Positions


In [3]:
acurrent = simdata.ExpansionFactor
zcurrent = simdata.Redshift
CurrentHubbleCt = 100. * simdata.HubbleParam *  np.sqrt(1. + simdata.Omega0*(1./acurrent-1.) + simdata.OmegaLambda* (acurrent**2-1.)) /acurrent
boxkms = simdata.BoxSize / simdata.HubbleParam * acurrent * CurrentHubbleCt

vpixsizekms = 1
nveloc = int(boxkms / vpixsizekms) + 1

In [4]:
Coordinates_conv     = convert.Pos_aexp_exp * np.log10(simdata.ExpansionFactor) + convert.Pos_h_exp * np.log10(simdata.HubbleParam) + np.log10(convert.Pos_cgs_unit) - np.log10(convert.cm_per_mpc) # in physical Mpc
Coordinates_conv     = 10.0**Coordinates_conv
Position = loss.Position * Coordinates_conv
BoxPhys = simdata.BoxSize * Coordinates_conv
ParticleSmoothingLength = loss.ParticleSmoothingLength * Coordinates_conv

In [5]:
Velocity_conv = convert.Vel_aexp_exp * np.log10(simdata.ExpansionFactor) + convert.Vel_h_exp * np.log10(simdata.HubbleParam) + np.log10(convert.Vel_cgs_unit) - np.log10(1e5) #$ physical km/s
Velocity_conv = 10.0**Velocity_conv
Velocity = loss.Velocity * Velocity_conv

In [6]:
Density_conv = convert.Dens_aexp_exp * np.log10(simdata.ExpansionFactor) + convert.Dens_h_exp * np.log10(simdata.HubbleParam) + np.log10(convert.Dens_cgs_unit)- np.log10(convert.proton_mass)
Density_conv = 10**Density_conv
ParticleDensity = loss.ParticleDensity * Density_conv
#Convert from total density to *Hydrogen* number density
ParticleDensity = ParticleDensity * loss.MassFractions[:,0]
Metallicity = loss.Metallicity[...] / const.Zmass_solar  #metallicity in solar units


In [7]:
Mass_conv = convert.Mass_aexp_exp * np.log10(simdata.ExpansionFactor) + convert.Mass_h_exp * np.log10(simdata.HubbleParam) + np.log10(convert.Mass_cgs_unit) - np.log10(convert.solar_mass)
Mass_conv = 10.0**Mass_conv
Mass   = loss.Mass * Mass_conv

In [8]:
Temp_Conv           = convert.Temp_aexp_exp * np.log10(simdata.ExpansionFactor) + convert.Temp_h_exp * np.log10(simdata.HubbleParam) + np.log10(convert.Temp_cgs_unit) #! cgs
Temp_Conv           = 10.0**Temp_Conv
ParticleTemperature = loss.ParticleTemperature * Temp_Conv 

In [9]:
x_physical = np.average(Position[:,0])
y_physical = np.average(Position[:,1])

In [11]:
los_long_axis

0

In [12]:
Position_2 = np.zeros(np.shape(Position))

if los_long_axis == 0:
    Position_2[:,0] = Position[:,1]
    Position_2[:,1] = Position[:,2]
    Position_2[:,2] = Position[:,0]
elif los_long_axis == 1:
    Position_2[:,0] = Position[:,2]
    Position_2[:,1] = Position[:,0]
    Position_2[:,2] = Position[:,1]
    
else:
    Position_2[:,0] = Position[:,0]
    Position_2[:,1] = Position[:,1]
    Position_2[:,2] = Position[:,2]


In [13]:
x_physical = np.average(Position_2[:,0])
y_physical = np.average(Position_2[:,1])

In [14]:

ions_to_do = [parameters.doH1,parameters.doHe2,parameters.doC2,
parameters.doC3,parameters.doC4,parameters.doN2,parameters.doN3,parameters.doN4,
parameters.doN5,parameters.doO1,parameters.doO3,parameters.doO4,parameters.doO5,
parameters.doO6,parameters.doO7,parameters.doMg2,parameters.doNe8,parameters.doAl2,
parameters.doAl3,parameters.doSi2,parameters.doSi3,parameters.doSi4,parameters.doS5,
parameters.doFe2,parameters.doFe3,parameters.do21cm]

number_ions = np.sum(ions_to_do) # was nion in the original F90 subroutine, prob superflues in this implimentation

number_species =  number_ions# was nspecies in the original F90

if (number_ions==0):
    print("ERROR: No ions were selected!")
    sys.exit()
if (parameters.doH1 and parameters.do_long_spectrum and parameters.nlyman > const.nlyman_all):
    print('ERROR: nlyman > n_lines_max')
    sys.exit()
if (const.Lambda_H1[0] !=  const.lyalpha):    # a bit superflus
    print( 'ERROR: Lambda_H1[0] and lyalpha  are not equal ')

ionbal_names = np.array(["h1","he2","c2","c3","c4","n2","n3","n4","n5","o1","o3","o4","o5","o6",  #The elements of this list must be in the same order as
                "o7","mg2","ne8","al2","al3","si2","si3","si4","s5","fe2","fe3","21cm"])          # ions_to_do



In [15]:
ionbal_to_use = ionbal_names[ions_to_do]

In [16]:
ionbal_to_use

array(['h1', 'n2', 'n5', 'ne8', 'si2'], dtype='<U4')

In [17]:
# something like this...
ionss = []
first_iteration = True

for ion_file in ionbal_to_use:
    with h5.File(parameters.ibdir+ion_file+".hdf5","r") as f:
        if first_iteration:
            z_ranges_table = f["/redshift"][...]
            logt_table = f["/logt"][...]
            logd_table = f["/logd"][...]
            first_iteration = False

        ionss.append(f["/ionbal"][...].flatten())
        print(ion_file+' Ionization table loaded')


h1 Ionization table loaded
n2 Ionization table loaded
n5 Ionization table loaded
ne8 Ionization table loaded
si2 Ionization table loaded


In [18]:
ionbal_to_use

array(['h1', 'n2', 'n5', 'ne8', 'si2'], dtype='<U4')

In [19]:
ElementAtomicMass = [const.atom_dic[item][0] for item in ionbal_to_use]

In [20]:
MassFraction_indx = [const.atom_dic[item][-1] for item in ionbal_to_use]

In [33]:
loss.MassFractions[500,MassFraction_indx] * loss.Mass[500] / ElementAtomicMass[:]

array([5.50501816e+19, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00])

In [22]:
ParticleNeutralHFraction = loss.ParticleNeutralHFraction
ParticleMolecularHFraction = loss.ParticleMolecularHFraction

In [23]:
NGas = len(Mass)
ib_logt = logt_table
ib_logd = logd_table
ib_redshift =z_ranges_table
ionizbal = np.log10(ionss)
ion_elnr = MassFraction_indx 
nion = len(ionbal_to_use)
nz = len(z_ranges_table)
nznt = nz*len(ib_logt)

<ipython-input-23-61848e83ba78>:5: RuntimeWarning: divide by zero encountered in log10
  ionizbal = np.log10(ionss)


In [24]:
n_ion     =  np.zeros((nion,nveloc))

In [25]:
#blah = projectdata(Position, Velocity, ParticleSmoothingLength, ParticleDensity, ParticleTemperature, ParticleNeutralHFraction, ParticleMolecularHFraction,
#Metallicity ,ib_redshift, ionizbal, loss.MassFractions , Mass, ElementAtomicMass , ion_elnr, x_physical, y_physical,zcurrent,simdata.BoxSize, simdata.ExpansionFactor, ib_logd, ib_logt,simdata.HubbleParam, nveloc, NGas, nion, nz, nznt)

In [26]:
#here there could be an issue with the difference of indexing between python and fortran 
if (nz > 1):
    if (not parameters.use_maxdens_above_zmax):
        if (zcurrent < ib_redshift[0] or zcurrent > ib_redshift[-1]):
            print("ERROR: Z out of bounds of the ion table!")
            sys.exit()
    if(parameters.use_maxdens_above_zmax > ib_redshift[-1] and zcurrent > ib_redshift[-1]):
        iz1 = nz -1
        iz2 = nz
        dz1 = 0
        dz2 = 1.0
    else:
        iz2=1
        while(zcurrent > ib_redshift[iz2]):
            iz2 = iz2 =1
        iz1 = iz2-1
        dz1 = (ib_redshift[iz2] - zcurrent) / (ib_redshift[iz2] - ib_redshift[iz1])
        dz2 = 1. -dz1

In [45]:
zmingrid = 0
zmaxgrid = (simdata.BoxSize*simdata.ExpansionFactor) / simdata.HubbleParam
box      = zmaxgrid
dzgrid   = (zmaxgrid - zmingrid) / nveloc
dzinv    = 1. / dzgrid 
box_2    = 0.5 * box
densscale = (simdata.ExpansionFactor/simdata.ExpansionFactor)**3
ncontr= 0

In [53]:
ionh = []

for i in range(NGas):
  ##    cdef double[:] p       /* (how to define this in cpython) */

    xx      = Position_2[i,0]
    yy      = Position_2[i,1]
    zz      = Position_2[i,2]

    hh      = ParticleSmoothingLength[i]
    h2      = hh*hh
    hinv2   = 1. / h2
    hinv3   = hinv2 / hh

    dx      = abs(xx - x_physical)
    dy      = abs(yy - y_physical)

    if(dx > box_2):
        dx  = box - dx

    if(dy > box_2):
        dy = box - dy

    b2    = dx*dx + dy*dy
    b     = (b2)**0.5
    impactparameter = b

    if( impactparameter <= hh):

    #    print(i)
    #    print(hh)
        ncontr = ncontr + 1

        #Density

        Density = ParticleDensity[i]

        if(Density < 0):
            print("ERROR:  Negative Density")

        Density = Density * densscale
        log_dens = np.log10(Density)

        log_temp = np.log10(ParticleTemperature[i])
        ionfrac = computeib(ib_redshift[iz1],ib_redshift[iz2],iz1, iz2, dz1, dz2, log_temp, log_dens, ib_logd, ionizbal, ib_logt, nz, nznt)
        ionh.append(ionfrac[0])
        
        #TO DO: URCHIN
        
        if parameters.aurora:
            ionfrac[0] = loss.ParticleNeutralHFraction[i]
        #TO DO : Ioncracone
        
        #Peculiar Velocities
        if parameters.NoPecVel:
            vr = 0
        else:
            vr = Velocity[i,los_long_axis]

        #Central Pixel 
        iz = int((zz - zmingrid) * dzinv + 1)
        
        #contribute to projection segment

        dzmax = np.sqrt(np.abs(h2 - b2))
        ioff  = int(dzmax * dzinv) + 2
        
        #To Do: gimic (?)
        
        if(ioff*2 > nveloc):
            print("Error ioff*2 > nveloc")
        
        if 
        break
        
        
        
        

In [58]:
iiz = iz-ioff
iizg = iz+ioff+1
print(iiz,iizg)

1168 1173


In [61]:
j = np.mod(iiz-1+10*nveloc,nveloc)+1

In [63]:
zzgird = zmingrid + (j-0.5)*dzgrid

In [66]:
deltaz = abs(zzgird - zz)
print(deltaz)

0.03655829089890261


In [68]:
if (deltaz > box_2):
    deltaz = box - deltaz

In [69]:
dr2 = b2 + deltaz**2

In [70]:
zf = deltaz + dzgrid*0.5
zi = deltaz - dzgrid*0.5

In [71]:
print(zf,zi)

0.04390131496608995 0.02921526683171528


In [72]:
q = np.sqrt(dr2 * hinv2) # q = r/h 
print(q)

1.0285145178314254


In [ ]:
if (q < 0.5):
    kernel_factor = (1.+6*q**2*(q-1))
elif ( q < 1):
    kernel_factor = 2.*(1-q)**3
else:
    kernel_factor = 0 
kernel_factor = (kernel_factor * 8. * hinv3) / np.pi

TO DO: URCHIN, AURORA 

In [ ]:
if parameters.aurora:
    ionfrac[0]

In [ ]:
(Position,Velocity,ParticleSmoothingLength, ParticleDensity, ParticleTemperature, ParticleNeutralHFraction,ParticleMolecularHFraction,
Metallicity ,ib_redshift, ionizbal,  MassFractions , Mass, ElementAtomicMas  s, ion_elnr, x_physical, y_physical, zcurrent, BoxSize, ExpansionFactor, ib_logd, ib_logt,  HubbleParam,
nveloc, NGas, nion, nz, nznt):

In [ ]:
#! Convert units, specwizard assumes the following units:
# Position:          Physical Mpc (and ParticleSmoothingLength too)
# Velocity:          Physical km/s
# ParticleTemperature:       K
# Density:           Hydrogen density in particles cm^-3
  !
  if (read_part_ids_from_file) then
     !
     !Open partid.hdf5:	
     call hdf5_open_file(part_file_handle, particle_file_name, readonly=.true.)
     !
     !read data from partid.hdf5 file:
     call hdf5_read_data(part_file_handle, 'MyData', partid)
     !
     call hdf5_close_file(part_file_handle)
     !
     call modify_metallicity_of_flagged_particles()
     !
  endif
  !
 
  !
  if(HubbleParam .eq. 0) call abortrun('HubbleParam = 0!')
  Temp_conv          = Temp_aexp_exp * log10(ExpansionFactor) + Temp_h_exp * log10(HubbleParam) + log10(Temp_cgs_unit) ! cgs
  Temp_Conv          = 10.d0**Temp_Conv
  ParticleTemperature        = ParticleTemperature * Temp_Conv 
  !
  if(setmaxt4sfgas) then
    where(StarFormationRate .gt. 0) ParticleTemperature = 1.d4
  endif
  !
  if(ignore_starforming) then
    where(StarFormationRate .gt. 0) Mass = 0.d0 ! Particles on the EOS do not contribute
  endif
  !
  MetallicityinSolar = Metallicity / zmass_solar ! convert metallicity to solar values
  if(MyPE == 0) &
       write(*,*) ' Snapshot read'